## Titanic Data Science Solutions

> [Kaggle link](https://www.kaggle.com/c/titanic)

https://colab.research.google.com/github/EmaSuriano/python-demos/blob/main/data-science/titanic.ipynb


### The Challenge

The sinking of the Titanic is one of the most infamous shipwrecks in history.

On April 15, 1912, during her maiden voyage, the widely considered “unsinkable” RMS Titanic sank after colliding with an iceberg. Unfortunately, there weren’t enough lifeboats for everyone onboard, resulting in the death of 1502 out of 2224 passengers and crew.

While there was some element of luck involved in surviving, it seems some groups of people were more likely to survive than others.

In this challenge, we ask you to build a predictive model that answers the question: “what sorts of people were more likely to survive?” using passenger data (ie name, age, gender, socio-economic class, etc).


### Dataset Description

The data has been split into two groups:

- training set (train.csv)
- test set (test.csv)

**The training set** should be used to build your machine learning models. For the training set, we provide the outcome (also known as the “ground truth”) for each passenger. Your model will be based on “features” like passengers’ gender and class. You can also use feature engineering to create new features.

**The test set** should be used to see how well your model performs on unseen data. For the test set, we do not provide the ground truth for each passenger. It is your job to predict these outcomes. For each passenger in the test set, use the model you trained to predict whether or not they survived the sinking of the Titanic.

We also include **gender_submission.csv**, a set of predictions that assume all and only female passengers survive, as an example of what a submission file should look like.

#### Data Dictionary

| Variable | Definition                                 | Key                                            |
| -------- | ------------------------------------------ | ---------------------------------------------- |
| survival | Survival                                   | 0 = No, 1 = Yes                                |
| pclass   | Ticket                                     | class 1 = 1st, 2 = 2nd, 3 = 3rd                |
| sex      | Sex                                        |
| Age      | Age in years                               |
| sibsp    | # of siblings / spouses aboard the Titanic |
| parch    | # of parents / children aboard the Titanic |
| ticket   | Ticket number                              |
| fare     | Passenger fare                             |
| cabin    | Cabin number                               |
| embarked | Port of Embarkation                        | C = Cherbourg, Q = Queenstown, S = Southampton |

#### Variable Notes

- pclass: A proxy for socio-economic status (SES)
  - 1st = Upper
  - 2nd = Middle
  - 3rd = Lower
- age: Age is fractional if less than 1. If the age is estimated, is it in the form of xx.5
- sibsp: The dataset defines family relations in this way...
  - Sibling = brother, sister, stepbrother, stepsister
  - Spouse = husband, wife (mistresses and fiancés were ignored)
- parch: The dataset defines family relations in this way...
  - Parent = mother, father
  - Child = daughter, son, stepdaughter, stepson
  - Some children travelled only with a nanny, therefore parch=0 for them.


### 1. Set up Environment


In [311]:
# data analysis and wrangling
import pandas as pd
import numpy as np

# visualization
import seaborn as sns
import matplotlib.pyplot as plt

In [312]:
import sys


def load_df(path: str):
    local_path = f"./ds/{path}"
    remote_path = (
        f"https://cdn.jsdelivr.net/gh/emasuriano/python-demos/data-science/ds/{path}"
    )

    IN_COLAB = "google.colab" in sys.modules
    path_to_csv = remote_path if IN_COLAB else local_path

    # Load dataset using pandas
    return pd.read_csv(path_to_csv)


# Load dataset using pandas
df = load_df("titanic/train.csv")

### Step 2. Understand your Data


In [313]:
df.shape

(891, 12)

In [314]:
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [315]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


### Step 3. Data Cleaning


In [316]:
# check for duplicates
df.duplicated().sum()

0

In [317]:
# check for null values
df.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

### Step 4. Data Transformation


In [318]:
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline
from sklearn.discriminant_analysis import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier


# Define columns
numerical_cols = ["Age", "Fare", "SibSp", "Parch"]
categorical_cols = ["Sex", "Embarked"]

numerical_transformer = Pipeline(
    steps=[
        ("imputer", SimpleImputer(strategy="mean")),
        ("scaler", StandardScaler()),
    ]
)

categorical_transformer = Pipeline(
    steps=[
        ("imputer", SimpleImputer(strategy="most_frequent")),
        ("onehot", OneHotEncoder(handle_unknown="ignore")),
    ]
)

# Define column transformer
preprocessor = ColumnTransformer(
    transformers=[
        ("num", numerical_transformer, numerical_cols),
        ("cat", categorical_transformer, categorical_cols),
    ],
    remainder="drop",
)

### Identify model


In [319]:
X, y = (df.drop(["Survived"], axis=1), df["Survived"])

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=10)

In [320]:
# Define models to evaluate
models = {
    "Logistic Regression": LogisticRegression(max_iter=200, solver="liblinear"),
    "Decision Tree": DecisionTreeClassifier(),
    "Random Forest": RandomForestClassifier(),
    "Gradient Boosting": GradientBoostingClassifier(),
    "SVM": SVC(),
    "K-Nearest Neighbors": KNeighborsClassifier(),
}

for name, model in models.items():
    pipeline = Pipeline(steps=[("preprocessor", preprocessor), ("classifier", model)])
    scores = cross_val_score(pipeline, X, y, cv=5, scoring="accuracy")
    print(
        f"{name}: Mean CV Accuracy = {np.mean(scores):.4f} (+/- {np.std(scores):.4f})"
    )

Logistic Regression: Mean CV Accuracy = 0.7879 (+/- 0.0193)
Decision Tree: Mean CV Accuracy = 0.7688 (+/- 0.0178)
Random Forest: Mean CV Accuracy = 0.8003 (+/- 0.0296)
Gradient Boosting: Mean CV Accuracy = 0.8182 (+/- 0.0115)
SVM: Mean CV Accuracy = 0.8272 (+/- 0.0153)
K-Nearest Neighbors: Mean CV Accuracy = 0.7969 (+/- 0.0138)


## Tuning parameters and get best estimator


In [321]:
from sklearn.model_selection import GridSearchCV


pipeline = Pipeline(
    [
        ("preprocessor", preprocessor),
        ("classifier", SVC()),
    ]
)

param_grid = {
    "classifier__C": [0.1, 1, 10],
    "classifier__kernel": ["linear", "rbf"],
    "classifier__gamma": ["scale", 0.01, 0.1],
}

gs = GridSearchCV(pipeline, param_grid, cv=5, scoring="accuracy", verbose=1)
gs.fit(X_train, y_train)

Fitting 5 folds for each of 18 candidates, totalling 90 fits


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('preprocessor',
                                        ColumnTransformer(transformers=[('num',
                                                                         Pipeline(steps=[('imputer',
                                                                                          SimpleImputer()),
                                                                                         ('scaler',
                                                                                          StandardScaler())]),
                                                                         ['Age',
                                                                          'Fare',
                                                                          'SibSp',
                                                                          'Parch']),
                                                                        ('cat',
                                                                         Pipeline(steps=[('imputer',
                                                                                          SimpleImputer(strategy='most_frequent')),
                                                                                         ('onehot',
                                                                                          OneHotEncoder(handle_unknown='ignore'))]),
                                                                         ['Sex',
                                                                          'Embarked'])])),
                                       ('classifier', SVC())]),
             param_grid={'classifier__C': [0.1, 1, 10],
                         'classifier__gamma': ['scale', 0.01, 0.1],
                         'classifier__kernel': ['linear', 'rbf']},
             scoring='accuracy', verbose=1)

In [322]:
final_model = gs.best_estimator_

gs.best_params_

{'classifier__C': 1, 'classifier__gamma': 'scale', 'classifier__kernel': 'rbf'}

In [323]:
from sklearn.metrics import accuracy_score

y_pred = final_model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)

print(f"Accuracy: {accuracy:.2f}")

Accuracy: 0.84


### Step Using the real data


In [324]:
X_final = load_df("titanic/test.csv")

y_final_predict = final_model.predict(X_final)

df_final = pd.DataFrame(
    {
        "PassengerId": X_final["PassengerId"],
        "Survived": y_final_predict,
    }
)

print(df_final)

     PassengerId  Survived
0            892         0
1            893         1
2            894         0
3            895         0
4            896         1
..           ...       ...
413         1305         0
414         1306         1
415         1307         0
416         1308         0
417         1309         0

[418 rows x 2 columns]


In [325]:
path = "./ds/titanic/submission.csv"
df_final.to_csv(path, index=False)

print(f"Results saved at {path}")

Results saved at ./ds/titanic/submission.csv
